<a href="https://colab.research.google.com/github/hjsonghjsong/best-repo-ever/blob/master/Auto_sk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install necessary pacakges by hjsong

In [3]:

!sudo apt-get install build-essential swig
!pip install auto-sklearn==0.11.1
!pip install dask distributed --upgrade  
!pip list

#!curl https://raw.githubusercontent.com/automl/auto-sklearn/master/requirements.txt | xargs -n 1 -L 1 pip install
#!pip install -U imbalanced-learn
#please restart runtime after you execute this cell.


Reading package lists... Done
Building dependency tree       
Reading state information... Done
build-essential is already the newest version (12.4ubuntu1).
Suggested packages:
  swig-doc swig-examples swig3.0-examples swig3.0-doc
The following NEW packages will be installed:
  swig swig3.0
0 upgraded, 2 newly installed, 0 to remove and 16 not upgraded.
Need to get 1,100 kB of archives.
After this operation, 5,822 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 swig3.0 amd64 3.0.12-1 [1,094 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 swig amd64 3.0.12-1 [6,460 B]
Fetched 1,100 kB in 1s (963 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 2.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (Thi

In [ ]:
!pip install --upgrade distributed 
#!pip uninstall scikit-learn
#!pip install scikit-learn==0.22.0

Requirement already up-to-date: distributed in /usr/local/lib/python3.6/dist-packages (2020.12.0)


In [3]:
import pandas as pd
import numpy as np
#import imblearn
import sklearn
#import math
#import matplotlib.pyplot as plt
#import matplotlib.pyplot as plt
import sklearn.metrics as metrics

from numpy import sqrt
from numpy import argmax
#from sklearn.ensemble import RandomForestClassifier
#from sklearn.ensemble import GradientBoostingClassifier
#from sklearn.svm import SVC
#from xgboost import XGBClassifier
#from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from collections import Counter

from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, StratifiedKFold
from collections import defaultdict
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_squared_error
from sklearn.metrics import classification_report
from sklearn.utils import resample
#from imblearn.over_sampling import SMOTE, ADASYN, BorderlineSMOTE, KMeansSMOTE, RandomOverSampler, SMOTENC, SVMSMOTE
#from imblearn.combine import SMOTEENN, SMOTETomek


#from itertools import cycle

from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
#from sklearn.preprocessing import label_binarize
#from sklearn.multiclass import OneVsRestClassifier
#from scipy import interp
#import shap



In [4]:
from google.colab import drive
drive.mount ('/drive')

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


In [5]:
#data = pd.read_csv("/drive/MyDrive/Colab Notebooks/")
data = pd.read_csv("/drive/MyDrive/Colab Notebooks/UKhyoi/895/data_class.csv")
data = data.drop(['Unnamed: 0', 'SEQN'], axis=1)
data

,RIAGENDR,RIDAGEYR,DMDHHSIZ,INDFMPIR,BMXBMI,DSD010,DSD010AN,SMD415,PAD590,PAD600,HUQ010,restaurant,protein,healthy,unhealthy,beverage,milk,MCQ010,MCQ053,MCQ092,MCQ140,active
0,1.0,19.0,2.0,2.44,50.85,0.0,0.0,0.0,4.0,3.0,3.0,0.0,82.0,73.0,108.0,3.0,10.0,0.0,0.0,0.0,0.0,1
1,2.0,16.0,6.0,2.47,20.78,0.0,0.0,1.0,5.0,1.0,2.0,2.0,100.0,94.0,137.0,3.0,10.0,0.0,0.0,0.0,0.0,0
2,2.0,14.0,5.0,1.60,18.43,0.0,0.0,1.0,4.0,2.0,2.0,2.0,62.0,92.0,95.0,4.0,14.0,0.0,0.0,0.0,0.0,0
3,1.0,17.0,7.0,2.75,20.65,0.0,0.0,1.0,4.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0
4,1.0,55.0,2.0,3.79,31.26,0.0,0.0,0.0,2.0,3.0,2.0,9.0,87.0,111.0,121.0,7.0,13.0,0.0,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12563,1.0,19.0,2.0,0.66,19.25,0.0,0.0,0.0,3.0,3.0,3.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
12564,1.0,12.0,5.0,0.35,18.41,0.0,0.0,0.0,4.0,1.0,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
12565,1.0,34.0,2.0,3.66,26.23,1.0,0.0,0.0,5.0,2.0,2.0,9.0,90.0,85.0,133.0,17.0,19.0,0.0,0.0,0.0,0.0,0
12566,1.0,21.0,6.0,1.16,26.16,0.0,0.0,0.0,2.0,0.0,4.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


# Train-Test Split

In [ ]:
X_data = data.drop('active', axis=1)
y_data = data['active']
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, shuffle=True, random_state=27)
print(X_data.shape)
print(Counter(y_data))
print(X_train.shape)
print(Counter(y_train))
print(X_test.shape)
print(Counter(y_test))

(12568, 21)
Counter({0: 9942, 1: 2626})
(10054, 21)
Counter({0: 7973, 1: 2081})
(2514, 21)
Counter({0: 1969, 1: 545})


## **auto-sklearn**

**WITHOUT threasholding
Test matrix : Test matrix : sensitivity, specificity, gmean, auc** **굵은 텍스트**

In [ ]:
import autosklearn.classification
import math

cls = autosklearn.classification.AutoSklearnClassifier()  #time_left_for_this_task=31
cls.fit(X_train, y_train)
print(cls.show_models())
#predictions = cls.predict(X_test)

y_predicted = cls.predict(X_test)
y_preds = cls.predict_proba(X_test)
preds = y_preds[:,1]



fpr, tpr, _ = metrics.roc_curve(y_test, preds)

auc_score = metrics.auc(fpr, tpr)

tn, fp, fn, tp = confusion_matrix(y_test, y_predicted).ravel()
sensitivity = tp / (tp+fn)
specificity = tn / (tn+fp)
g_mean = math.sqrt(specificity*sensitivity)

print("g_mean: ", g_mean)
print("auc: ", auc_score)
print("sensitivity: ", sensitivity)
print("specificity: ", specificity)
   


/usr/local/lib/python3.6/dist-packages/distributed/node.py:155: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 43897 instead
  http_address["port"], self.http_server.port


[WARNING] [2021-01-07 05:42:34,511:AutoML(1):0a7a65b85fa4d625d09e947de0ee71c4] No valid ensemble was created. Please check the logfile for errors. Default to the best individual estimator:[(1, 2, 0.0)]
[(1.000000, SimpleClassificationPipeline({'balancing:strategy': 'none', 'classifier:__choice__': 'random_forest', 'data_preprocessing:categorical_transformer:categorical_encoding:__choice__': 'one_hot_encoding', 'data_preprocessing:categorical_transformer:category_coalescence:__choice__': 'minority_coalescer', 'data_preprocessing:numerical_transformer:imputation:strategy': 'mean', 'data_preprocessing:numerical_transformer:rescaling:__choice__': 'standardize', 'feature_preprocessor:__choice__': 'no_preprocessing', 'classifier:random_forest:bootstrap': 'True', 'classifier:random_forest:criterion': 'gini', 'classifier:random_forest:max_depth': 'None', 'classifier:random_forest:max_features': 0.5, 'classifier:random_forest:max_leaf_nodes': 'None', 'classifier:random_forest:min_impurity_decre

# **With threasholding,  metrix : auc, gmean, specificity and sensitivity**

In [2]:
y_preds = cls.predict_proba(X_test)
preds = y_preds[:,1]
fpr, tpr, thresholds = metrics.roc_curve(y_test, preds)

auc_score = metrics.auc(fpr, tpr)
# auc_score = roc_auc_score(y_test, preds)

# calculate the g-mean for each threshold
gmeans = sqrt(tpr * (1-fpr))
# locate the index of the largest g-mean
ix = argmax(gmeans)

print("threshold: ", thresholds[ix])
print("auc: ", auc_score)
print("g_mean: ", gmeans[ix])
print("sensitivity: ", tpr[ix])
print("specificity: ", 1-fpr[ix])

NameError: ignored